In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install simplet5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 65.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 43.0 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6853 sha256=3d35bd8e6d8282b0d81ff8c91e4c51b629356be16fa11d94c68b7b6ed9dc025f
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
Successfully built simplet5
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at 

In [3]:
!pip install -q -U  datasets accelerate peft trl bitsandbytes

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simplet5 0.1.4 requires transformers==4.16.2, but you have transformers 4.37.2 which is incompatible.


In [4]:
from datasets import load_dataset, Dataset, DatasetDict
dataset = load_dataset("mayank200456789/Viva")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 381
    })
})

In [16]:
def remove_blank_strings(dataset):
    return dataset.filter(lambda example: example['text'] != "")

# Apply the function to the 'train' split
dataset["train"] = remove_blank_strings(dataset["train"])

# Print the modified dataset
print(DatasetDict(dataset))

Filter:   0%|          | 0/221 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 221
    })
})


In [17]:
def remove_blank_strings(dataset):
    return dataset.filter(lambda example: example['text'] != "")

# Apply the function to the 'train' split
dataset["train"] = remove_blank_strings(dataset["train"])

# Print the modified dataset
print(DatasetDict(dataset))

Filter:   0%|          | 0/221 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 221
    })
})


In [15]:
import pandas as pd

df = pd.DataFrame({
    "instruction": dataset["train"]["instruction"],
    "output": dataset["train"]["output"]
})

df.to_csv("dataset.csv", index=False)

KeyError: "Column instruction not in the dataset. Current columns in the dataset: ['text']"

In [14]:
import pandas as pd

df = pd.DataFrame({
    "instruction": dataset["train"]["instruction"],
    "output": dataset["train"]["output"]
})

df.to_csv("dataset.csv", index=False)

KeyError: "Column instruction not in the dataset. Current columns in the dataset: ['text']"

In [12]:
df_train=df

NameError: name 'df' is not defined

In [ ]:
df_train = df_train.rename(columns={"output":"target_text", "instruction":"source_text"})
df_train = df_train[['source_text', 'target_text']]
 
# T5 model expects a task related prefix: since it is a question answering task, we will add a prefix "answer: "
df_train['source_text'] = "answer: " + df_train['source_text']

In [8]:
from simplet5 import SimpleT5
 
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")from simplet5 import SimpleT5
 
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

SyntaxError: invalid syntax (11971172.py, line 4)

In [18]:
dataset = {
    "train": Dataset.from_dict({
        "instruction": dataset["train"]["text"][:220:2],
        "output": dataset["train"]["text"][1::2]
    })
}


print(DatasetDict(dataset))

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 110
    })
})


In [19]:
import pandas as pd

df = pd.DataFrame({
    "instruction": dataset["train"]["instruction"],
    "output": dataset["train"]["output"]
})

df.to_csv("dataset.csv", index=False)

In [21]:
df_train=df
df_train = df_train.rename(columns={"output":"target_text", "instruction":"source_text"})
df_train = df_train[['source_text', 'target_text']]
 
# T5 model expects a task related prefix: since it is a question answering task, we will add a prefix "answer: "
df_train['source_text'] = "answer: " + df_train['source_text']
# Display some data to see final data to finetune t5 model
# df_train.head()

In [22]:
df_train

,source_text,target_text
0,answer: What is Unit cell?,Smallest repeating structure in a crystal tha...
1,answer: What is Atomic basis?,Every point in space lattice is associated wit...
2,answer: What is Space lattice?,Infinite 3d array of points in which every poi...
3,answer: What is Susceptibility?,Magnetic susceptibility is the measure of exte...
4,answer: What are Diamagnetic materials?,Materials which weakly repell magnetic field a...
...,...,...
105,answer: Proximate Analysis,proximate Analysis gives approximate compositi...
106,answer: Ultimate Analysis,proximate Analysis gives approximate compositi...
107,answer: Moisture content,it controls the rate of combustion.
108,answer: Volatile matter,it helps in initial combustion.


In [23]:
from simplet5 import SimpleT5
 
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=df_train,
            eval_df=df_train[89:91],
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=16, max_epochs=25, use_gpu=True)

2024-02-09 17:00:54.614885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 17:00:54.615001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 17:00:54.740068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

OSError: [Errno 28] No space left on device

In [28]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
os.rmdir(folder_path)

OSError: [Errno 16] Device or resource busy: '/kaggle/working'

In [29]:
model.train(train_df=df_train,
            eval_df=df_train[80:82],
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=16, max_epochs=3, use_gpu=True,outputdir = 'trained_t5')

Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]